In [22]:
%pip install feedparser

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import feedparser
import pandas as pd
from datetime import datetime, timedelta
import ssl
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")

# SSL sertifika doğrulama sorununu çözmek için
if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

In [24]:
links = [
    "https://www.sciencedaily.com/rss/computers_math/artificial_intelligence.xml",
    "https://www.nature.com/subjects/medical-research.rss",  
    "https://www.technologyreview.com/feed/",  
    "https://medicalfuturist.com/feed/",  
    "https://venturebeat.com/category/ai/feed/", 
    "https://blogs.bmj.com/bmj/category/artificial-intelligence/feed/", 
    "https://www.healthcareitnews.com/taxonomy/term/8601/feed", 
    "https://www.frontiersin.org/journals/digital-health/rss"  
]

In [25]:
if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

feed = feedparser.parse("https://www.nature.com/subjects/medical-research.rss")

feed.entries

[{'title': 'Macrophage miR-4524a-5p/TBP promotes β-TrCP -TIM3 complex activation and TGFβ release and aggravates NAFLD-associated fibrosis',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://www.nature.com/subjects/medical-research.rss?error=cookies_not_supported&code=6b76c53c-cffd-46e5-b3c6-3a3e3fe268b9',
   'value': 'Macrophage miR-4524a-5p/TBP promotes β-TrCP -TIM3 complex activation and TGFβ release and aggravates NAFLD-associated fibrosis'},
  'published': 'Sat, 19 Apr 2025 00:00:00 +0000',
  'published_parsed': time.struct_time(tm_year=2025, tm_mon=4, tm_mday=19, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=5, tm_yday=109, tm_isdst=0),
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://www.nature.com/articles/s41419-025-07574-4'}],
  'link': 'https://www.nature.com/articles/s41419-025-07574-4',
  'summary': '',
  'summary_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://www.nature.com/subjects/medical-re

#### fetch papers

In [26]:
def fetch_feed(links):
    entries = {"Title": [], "Link": [], "Published": [], "Description": []}
    
    try:
        # Iterate over each RSS feed link
        for link in links:
            feed = feedparser.parse(link)
            
            # Process each entry/item in the feed
            for entry in feed.entries:
                entries["Title"].append(entry.get("title", "No Title"))
                entries["Link"].append(entry.get("link", "No Link"))
                entries["Published"].append(entry.get("published", "No Date"))
                entries["Description"].append(entry.get("description", "No Description"))
                
    except Exception as e:
        # Print error message if something goes wrong
        print(f"An error occurred: {e}")

    # Convert the collected entries into a DataFrame
    df = pd.DataFrame(entries)
    return df


In [29]:
def filter_medical_ai_content(df):
    # List of medical-related medical keywords 
    medical_keywords = [
        'medicine', 'medical', 'health', 'healthcare', 'clinical', 'hospital',
        'patient', 'disease', 'diagnosis', 'treatment', 'therapy', 'drug',
        'biomedical', 'doctor', 'physician', 'nursing', 'wellness', 'telemedicine',
        'pharmaceutical', 'mri', 'imaging', 'radiology', 'surgery', 'precision medicine',
        'genomics', 'pathology', 'oncology', 'cardiology', 'neurology'
    ]
    
    # Using regex to match any of the medical keywords in a case-insensitive manner
    medical_pattern = '|'.join(medical_keywords)
    
    # Filter articles that contain medical keywords in the title or description
    mask = (
        df['Title'].str.lower().str.contains(medical_pattern, na=False) |
        df['Description'].str.lower().str.contains(medical_pattern, na=False)
    )
    
    # Only keep articles with medical-related content
    df_medical_ai = df[mask].copy()
    
    return df_medical_ai

In [30]:
df = fetch_feed(links)

In [28]:
df.head(5)  # Display the first 5 rows of the DataFrame

,Title,Link,Published,Description
0,Macrophage miR-4524a-5p/TBP promotes β-TrCP -T...,https://www.nature.com/articles/s41419-025-075...,"Sat, 19 Apr 2025 00:00:00 +0000",
1,Helmholtz Health task force to strengthen prev...,https://www.nature.com/articles/s41591-025-035...,"Fri, 18 Apr 2025 00:00:00 +0000",
2,Artificial intelligence models utilize lifesty...,https://www.nature.com/articles/s41598-025-967...,"Fri, 18 Apr 2025 00:00:00 +0000",
3,Epileptic encephalopathy in a young Bengal cat...,https://www.nature.com/articles/s41598-025-984...,"Fri, 18 Apr 2025 00:00:00 +0000",
4,A comparison between radiomic biological age a...,https://www.nature.com/articles/s41598-025-982...,"Fri, 18 Apr 2025 00:00:00 +0000",


In [31]:
def extract_and_clean_data(df):
    try:
        # Regex pattern for date extraction
        pattern = r'(\d{2} \w{3} \d{4})'

        # Apply regex to the 'Published' column and extract the date
        df['date'] = df['Published'].str.extract(pattern)

        # Convert the extracted date to datetime
        df['date'] = pd.to_datetime(df['date'], format='%d %b %Y')

        # Drop the original 'Published' column
        df.drop(columns=['Published'], inplace=True)

        # Get today's date and calculate the date 7 days ago
        today = datetime.now()
        seven_days_ago = today - timedelta(days=7)

        # Filter the rows within the last 7 days
        df_last_seven_days = df[(df['date'] >= seven_days_ago) & (df['date'] <= today)]

        # Sort by date in descending order
        df_last_seven_days = df_last_seven_days.sort_values(by='date', ascending=False)

        # Function to clean HTML tags
        def clean_html(text):
            try:
                soup = BeautifulSoup(text, "html.parser")
                return soup.get_text()
            except Exception as e:
                print(f"Error cleaning HTML: {e}")
                return text

        # Apply the HTML cleaning function and shorten descriptions to 500 characters
        df_last_seven_days['Description'] = df_last_seven_days['Description'].apply(lambda x: clean_html(x)[:500])

        # Remove newline characters from the 'Description' column
        df_last_seven_days["Description"] = df_last_seven_days["Description"].str.replace("\n", "")

        return df_last_seven_days

    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

In [32]:
df_last_seven_days = extract_and_clean_data(df)

In [33]:
df_medical_ai = filter_medical_ai_content(df_last_seven_days)

In [34]:
df_medical_ai.sort_values(by='date', ascending=False, inplace=True)

In [35]:
print(f"There are {len(df_medical_ai)} papers in medicine:")
df_medical_ai

There are 16 papers in medicine:


,Title,Link,Description,date
23,The construction and validation of a predictio...,https://www.nature.com/articles/s41598-025-984...,,2025-04-18
24,Association between plausible genetic factors ...,https://www.nature.com/articles/s41591-025-036...,,2025-04-18
26,Delayed ambulation in adult patients after maj...,https://www.nature.com/articles/s41598-025-979...,,2025-04-18
27,Association between dietary selenium intake an...,https://www.nature.com/articles/s41598-025-978...,,2025-04-18
32,Longevity clinics around the world are selling...,https://www.technologyreview.com/2025/04/18/11...,"The quest for long, healthy life—and even immo...",2025-04-18
1,Helmholtz Health task force to strengthen prev...,https://www.nature.com/articles/s41591-025-035...,,2025-04-18
50,Identity as the new perimeter: National Oilwel...,https://venturebeat.com/security/nov-cio-fused...,NOV’s CIO led a cyber strategy fusing Zero Tru...,2025-04-18
20,Comparison between norepinephrine plus epineph...,https://www.nature.com/articles/s41598-025-968...,,2025-04-18
22,Automated pain detection using facial expressi...,https://www.nature.com/articles/s41598-025-978...,,2025-04-18
5,Association between triglyceride glucose index...,https://www.nature.com/articles/s41598-025-967...,,2025-04-18


In [39]:
df_medical_ai.to_csv('medical_ai_news.csv', index=False)